In [1]:
!pip install minio delta-spark==2.2.0

In [5]:
from minio import Minio
import io
import logging
import sys
import traceback
from datetime import datetime

import urllib3

from minio import Minio
from pyspark import SparkContext
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import unix_timestamp
import minio
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [6]:
spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.driver.host', 'f115dd85bf2e'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.2-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.2-bi

In [7]:
df_orders = spark.read.format("csv").option("delimiter", ";").load("s3a://warehouse/conso-elec-gaz-annuelle-par-naf-agregee-region.csv", header=True)
df_orders.limit(20).toPandas()

,operateur,annee,filiere,code_categorie_consommation,libelle_categorie_consommation,code_grand_secteur,libelle_grand_secteur,code_naf,libelle_secteur_naf2,conso,pdl,indqual,nombre_mailles_secretisees,code_region,libelle_region
0,Régie intercommunale d'électricité de Niederbr...,2021,Electricité,PRO,Petits professionels,A,Agriculture,None,0,115.362,9,0.491,0,44,Grand Est
1,Régie Municipale Bazas-Energies,2021,Electricité,RES,Résidentiel,R,Résidentiel,None,0,12471.471,2634,0.00721,0,75,Nouvelle-Aquitaine
2,Régie municipale d'électricité d'Arignac,2021,Electricité,ENT,Entreprises,T,Tertiaire,94,Activités des organisations associatives,493.0,1,1.0,0,76,Occitanie
3,Régie municipale d'électricité d'Arignac,2021,Electricité,PRO,Petits professionels,I,Industrie,None,0,0.0,0,0.0,1,76,Occitanie
4,Régie municipale d'électricité d'Arignac,2021,Electricité,PRO,Petits professionels,T,Tertiaire,None,0,385.0,52,1.0,0,76,Occitanie
5,Régie municipale d'électricité de l'Hospitalet...,2021,Electricité,ENT,Entreprises,T,Tertiaire,47,"Commerce de détail, à l'exception des automobi...",123.0,1,1.0,0,76,Occitanie
6,Régie municipale d'électricité de l'Hospitalet...,2021,Electricité,RES,Résidentiel,R,Résidentiel,None,0,256.0,83,1.0,0,76,Occitanie
7,Régie municipale d'électricité de La Bresse,2021,Electricité,ENT,Entreprises,T,Tertiaire,85,Enseignement,80.673,1,1.0,0,44,Grand Est
8,Régie municipale d'électricité de La Bresse,2021,Electricité,ENT,Entreprises,T,Tertiaire,93,"Activités sportives, récréatives et de loisirs",72.914,1,1.0,0,44,Grand Est
9,Régie municipale d'électricité de La Bresse,2021,Electricité,ENT,Entreprises,T,Tertiaire,96,Autres services personnels,613.007,1,0.867,0,44,Grand Est


In [8]:
def create_model(iter, features_cols, labelCol):
    # create LinearRegression estimator and set parameters
    lr = LinearRegression(maxIter=iter,
                          featuresCol=features_cols,
                          labelCol=labelCol)
    return lr


def set_train_and_validation_ds(data, seed):
    return data.randomSplit([0.7, 0.3], seed=seed)


def test_model(spark, model):
    # Methode pour tester le modèle avec un jeu de données test
    testData = spark.read.format("libsvm").load("path/to/test/file")
    predictions = model.transform(testData)
    predictions.show()


def save_model(model):
    # Methode pour sauvegarder le modèle vers un lieu spécifié
    model.save("saved_model/model_test")